# Clasificación de Sentimientos con una Red Neuronal Multicapa (MLP)

## Objetivo

En esta actividad vas a construir una red neuronal feedforward multicapa (MLP) usando PyTorch. El objetivo es entrenarla para que pueda clasificar frases en español como positivas o negativas.

### Con esto vas a:

- Comprender cómo se construye una red con múltiples capas y neuronas
- Usar funciones de activación no lineales (ReLU, Sigmoid)
- Implementar entrenamiento automático con optimizadores modernos
- Observar cómo una MLP mejora respecto al perceptrón simple del laboratorio anterior

### ¿Qué es una red neuronal multicapa?

A diferencia del perceptrón simple (una sola neurona), una MLP tiene:
- **Capa de entrada**: Recibe los features del texto
- **Capas ocultas**: Una o más capas intermedias que aprenden representaciones complejas
- **Capa de salida**: Produce la predicción final

Las capas ocultas permiten aprender patrones **no lineales**, lo que le da mucha más capacidad expresiva al modelo.

## 1. Preparación del entorno

Importamos PyTorch y NumPy para comenzar.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

print(f"PyTorch versión: {torch.__version__}")
print(f"Device disponible: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## 2. Datos de entrenamiento

Usamos un conjunto más grande de frases típicas de opiniones escritas en Argentina, etiquetadas como positivas (1) o negativas (0).

Vamos a incluir casos más variados y complejos que en el laboratorio anterior.

In [ ]:
frases = [
    "La verdad, este lugar está bárbaro. Muy recomendable",
    "Una porquería de servicio, nunca más vuelvo",
    "Me encantó la comida, aunque la música estaba muy fuerte",
    "El envío fue lento y el producto llegó dañado. Qué desastre",
    "Todo excelente. Atención de diez",
    "Qué estafa, me arrepiento de haber comprado",
    "Muy conforme con el resultado final",
    "No me gustó para nada la experiencia",
    "Superó mis expectativas, gracias",
    "No lo recomiendo, mala calidad"
]

etiquetas = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])  # 1 = Positivo, 0 = Negativo

print(f"Total de frases: {len(frases)}")
print(f"Balance: {sum(etiquetas)} positivas, {len(etiquetas) - sum(etiquetas)} negativas\n")
print("Ejemplos:")
for i in range(3):
    sentimiento = "Positivo" if etiquetas[i] == 1 else "Negativo"
    print(f"  {i+1}. '{frases[i]}' → {sentimiento}")

## 3. Construcción del vocabulario

Definimos manualmente un vocabulario con palabras que suelen aparecer en frases de opinión con carga positiva o negativa.

En este caso expandimos el vocabulario para cubrir más términos comunes.

In [ ]:
vocabulario = [
    "bárbaro", "recomendable", "porquería", "nunca", "encantó",
    "fuerte", "desastre", "excelente", "estafa", "arrepiento",
    "conforme", "gustó", "superó", "gracias", "recomiendo", "mala"
]

print(f"Vocabulario: {len(vocabulario)} palabras clave")
print(f"\nPalabras: {vocabulario}")

## 4. Preprocesamiento: vectorización de las frases

Seguimos usando bag-of-words como en el laboratorio anterior: cada frase se convierte en un vector binario que indica si contiene alguna de las palabras del vocabulario.

Luego convertimos estos vectores a tensores de PyTorch para poder usarlos con redes neuronales.

In [ ]:
def vectorizar(frase, vocabulario):
    """
    Convierte una frase en un vector binario según el vocabulario.
    
    Args:
        frase: String con la frase a vectorizar
        vocabulario: Lista de palabras clave
    
    Returns:
        Array de numpy con 1s y 0s (float32 para PyTorch)
    """
    tokens = frase.lower().split()
    return np.array([1 if palabra in tokens else 0 for palabra in vocabulario], dtype=np.float32)

# Vectorizamos todas las frases
X_np = np.array([vectorizar(frase, vocabulario) for frase in frases], dtype=np.float32)
y_np = etiquetas.astype(np.float32).reshape(-1, 1)

# Convertimos a tensores de PyTorch
X = torch.tensor(X_np)
y = torch.tensor(y_np)

print("Datos preprocesados:")
print(f"  X shape: {X.shape} (frases × features)")
print(f"  y shape: {y.shape} (frases × 1)")
print(f"\nPrimera frase vectorizada: {X[0]}")
print(f"Etiqueta: {y[0].item()}")

## 5. Definición del modelo MLP

Vamos a crear un modelo simple con:
- **Capa de entrada**: Tamaño = cantidad de palabras en el vocabulario
- **Capa oculta**: 8 neuronas con activación ReLU
- **Capa de salida**: 1 neurona con activación Sigmoid (para clasificación binaria)

### ¿Por qué estas activaciones?

- **ReLU** (Rectified Linear Unit): `f(x) = max(0, x)` → Introduce no linealidad, permite aprender patrones complejos
- **Sigmoid**: `f(x) = 1 / (1 + e^(-x))` → Convierte la salida a un valor entre 0 y 1 (probabilidad)

In [ ]:
###############################################################################
# CONFIGURACIÓN DE LA ARQUITECTURA: DEFINICIÓN DE LA RED NEURONAL MULTICAPA
###############################################################################

# ¿Qué es input_size?
# Es la dimensión de entrada de la red: cantidad de features que recibe.
# En este caso = 16 (tamaño del vocabulario)
# Cada frase se representa como un vector de 16 posiciones (bag-of-words)
input_size = len(vocabulario)

# ¿Qué es hidden_size y por qué 8?
# Es la cantidad de neuronas en la capa oculta.
#
# ¿Cómo elegir este número?
# No hay una fórmula exacta, es un hiperparámetro que se ajusta experimentalmente.
#
# Reglas prácticas:
# - Muy pequeño (ej: 2): La red no tiene capacidad para aprender patrones complejos
# - Muy grande (ej: 1000): Riesgo de overfitting, especialmente con pocos datos
# - Valor típico: Entre input_size/2 y input_size*2
#
# ¿Por qué 8 en este caso?
# - Es la mitad del input_size (16/2 = 8)
# - Suficiente para aprender patrones simples
# - No es excesivo para nuestro dataset pequeño (10 ejemplos)
#
# Analogía: Es como el número de "conceptos intermedios" que la red puede aprender
# Ej: Cada neurona oculta podría especializarse en detectar:
#   - Neurona 1: Palabras muy positivas (excelente, encantó)
#   - Neurona 2: Negaciones (no, nunca)
#   - Neurona 3: Palabras negativas (porquería, desastre)
#   - etc.
hidden_size = 8

###############################################################################
# DEFINICIÓN DE LA CLASE DEL MODELO
###############################################################################

class MLP(nn.Module):
    """
    Red Neuronal Multicapa (Multi-Layer Perceptron)
    
    Arquitectura:
    Input (16) → Hidden (8) con ReLU → Output (1) con Sigmoid
    
    ¿Por qué heredamos de nn.Module?
    Es la clase base de PyTorch para todos los modelos.
    Nos da funcionalidades automáticas como:
    - Gestión de parámetros
    - Mover el modelo a GPU
    - Cambiar entre modo train/eval
    - Serialización (guardar/cargar modelos)
    """
    
    def __init__(self):
        super(MLP, self).__init__()
        
        # ¿Qué hace nn.Sequential?
        # Encadena capas que se ejecutan en orden.
        # Equivale a: salida = Sigmoid(Linear2(ReLU(Linear1(entrada))))
        #
        # Ventaja: Código más limpio y legible
        # Alternativa: Definir cada capa por separado y llamarlas manualmente en forward()
        self.net = nn.Sequential(
            ###################################################################
            # CAPA 1: Linear (Capa oculta)
            ###################################################################
            # nn.Linear(input_size, hidden_size) crea una transformación lineal:
            # z = W·x + b
            #
            # Dimensiones:
            # - W (pesos): matriz de 8×16 (hidden_size × input_size)
            # - b (bias): vector de 8 valores
            #
            # ¿Cómo se inicializan estos parámetros?
            # PyTorch usa inicialización Kaiming/He por defecto:
            # - Pesos: distribución uniforme U(-k, k) donde k = sqrt(1/input_size)
            # - Bias: también U(-k, k)
            #
            # ¿Por qué esta inicialización?
            # - Evita que las activaciones exploten o se desvanezcan
            # - Mantiene la varianza de las activaciones estable entre capas
            #
            # Parámetros totales en esta capa: (16 × 8) + 8 = 136
            nn.Linear(input_size, hidden_size),
            
            ###################################################################
            # ACTIVACIÓN 1: ReLU (Rectified Linear Unit)
            ###################################################################
            # ReLU(x) = max(0, x)
            #
            # ¿Qué hace?
            # - Si x > 0: deja pasar el valor sin cambios
            # - Si x ≤ 0: lo convierte en 0
            #
            # Ejemplo: ReLU([-2, 0, 3]) = [0, 0, 3]
            #
            # ¿Por qué ReLU y no otra función?
            # Ventajas de ReLU:
            # - Computacionalmente eficiente (solo una comparación)
            # - No sufre de "vanishing gradient" como Sigmoid/Tanh
            # - Introduce no linealidad (permite aprender patrones complejos)
            # - Ha demostrado funcionar muy bien en la práctica
            #
            # Sin ReLU, la red sería equivalente a una regresión lineal
            # (dos capas lineales seguidas = una sola capa lineal)
            #
            # Alternativas populares:
            # - Leaky ReLU: f(x) = max(0.01x, x) (evita neuronas "muertas")
            # - GELU: usada en Transformers modernos
            # - Tanh: f(x) = (e^x - e^-x)/(e^x + e^-x)
            nn.ReLU(),
            
            ###################################################################
            # CAPA 2: Linear (Capa de salida)
            ###################################################################
            # Transforma de 8 dimensiones (hidden) a 1 dimensión (salida)
            #
            # Dimensiones:
            # - W: matriz de 1×8
            # - b: escalar (1 valor)
            #
            # Parámetros: (8 × 1) + 1 = 9
            nn.Linear(hidden_size, 1),
            
            ###################################################################
            # ACTIVACIÓN 2: Sigmoid
            ###################################################################
            # Sigmoid(x) = 1 / (1 + e^(-x))
            #
            # ¿Qué hace?
            # Convierte cualquier número real a un valor entre 0 y 1
            #
            # Ejemplos:
            # - Sigmoid(-10) ≈ 0.00005 (casi 0)
            # - Sigmoid(0) = 0.5
            # - Sigmoid(10) ≈ 0.99995 (casi 1)
            #
            # ¿Por qué Sigmoid en la salida?
            # - Interpretamos la salida como probabilidad: P(clase positiva)
            # - Rango [0, 1] es perfecto para clasificación binaria
            # - Si salida ≥ 0.5 → predecimos clase 1 (positivo)
            # - Si salida < 0.5 → predecimos clase 0 (negativo)
            #
            # ¿Por qué NO usamos ReLU acá?
            # ReLU no está acotada superiormente: ReLU(100) = 100
            # Necesitamos una salida entre 0 y 1 para interpretarla como probabilidad
            #
            # ¿Por qué NO usamos Softmax?
            # Softmax se usa para clasificación multiclase (>2 clases)
            # Para clasificación binaria, Sigmoid es más simple y eficiente
            nn.Sigmoid()
        )
    
    def forward(self, x):
        """
        Propagación hacia adelante (forward pass)
        
        ¿Qué hace esta función?
        Define cómo fluyen los datos desde la entrada hasta la salida.
        
        Flujo:
        1. x entra (batch_size × 16)
        2. Primera Linear: (batch × 16) @ (16 × 8) + (8) = (batch × 8)
        3. ReLU: aplica max(0, ·) elemento a elemento
        4. Segunda Linear: (batch × 8) @ (8 × 1) + (1) = (batch × 1)
        5. Sigmoid: convierte a probabilidades
        
        ¿Por qué se llama 'forward'?
        En entrenamiento también hay un 'backward' pass (backpropagation)
        donde se calculan gradientes en dirección opuesta.
        
        Args:
            x: Tensor de entrada (batch_size × input_size)
        
        Returns:
            Tensor de salida (batch_size × 1) con valores entre 0 y 1
        """
        return self.net(x)

# Creamos una instancia del modelo
modelo = MLP()

print("Arquitectura del modelo:")
print(modelo)

# ¿Cuántos parámetros entrenables tiene el modelo?
# Capa 1: 16×8 + 8 = 136
# Capa 2: 8×1 + 1 = 9
# Total: 145 parámetros
#
# Comparación:
# - Perceptrón simple: 16 + 1 = 17 parámetros
# - Esta MLP: 145 parámetros (8.5 veces más capacidad)
print(f"\nParámetros totales: {sum(p.numel() for p in modelo.parameters())}")

## 6. Configuración del entrenamiento

Necesitamos definir dos componentes clave:

### Función de pérdida (Loss Function)

Usamos **Binary Cross Entropy (BCE)**: mide qué tan diferentes son las predicciones del modelo de las etiquetas reales. El objetivo del entrenamiento es minimizar esta pérdida.

Fórmula: `BCE = -[y·log(ŷ) + (1-y)·log(1-ŷ)]`

### Optimizador

Usamos **Adam**: un optimizador moderno que ajusta automáticamente la tasa de aprendizaje para cada parámetro. Es más eficiente que el ajuste manual que hicimos en el perceptrón.

In [ ]:
###############################################################################
# CONFIGURACIÓN DEL ENTRENAMIENTO: FUNCIÓN DE PÉRDIDA Y OPTIMIZADOR
###############################################################################

###############################################################################
# FUNCIÓN DE PÉRDIDA: Binary Cross Entropy (BCE)
###############################################################################
# ¿Qué mide la función de pérdida?
# Cuantifica qué tan "equivocadas" son las predicciones del modelo.
#
# BCE (Binary Cross Entropy):
# Loss = -[y·log(ŷ) + (1-y)·log(1-ŷ)]
#
# Donde:
# - y = etiqueta real (0 o 1)
# - ŷ = predicción del modelo (entre 0 y 1)
#
# Ejemplo numérico:
# Si y=1 (clase positiva real) y ŷ=0.9 (modelo muy confiado en positivo):
#   BCE = -[1·log(0.9) + 0·log(0.1)]
#   BCE = -log(0.9)
#   BCE ≈ 0.105 (pérdida baja, predicción buena)
#
# Si y=1 (clase positiva real) y ŷ=0.1 (modelo predice negativo):
#   BCE = -[1·log(0.1) + 0·log(0.9)]
#   BCE = -log(0.1)
#   BCE ≈ 2.303 (pérdida alta, predicción mala)
#
# ¿Por qué BCE y no otra función?
# - Específicamente diseñada para clasificación binaria
# - Penaliza más las predicciones muy confiadas pero incorrectas
# - Tiene buenas propiedades matemáticas para optimización
# - Es diferenciable (permite calcular gradientes)
#
# Alternativas:
# - MSE (Mean Squared Error): (y - ŷ)²
#   Funciona, pero BCE converge más rápido en clasificación
# - Hinge Loss: Usada en SVMs
# - Focal Loss: Variante de BCE para datasets desbalanceados
criterio = nn.BCELoss()

###############################################################################
# OPTIMIZADOR: Adam (Adaptive Moment Estimation)
###############################################################################
# ¿Qué hace un optimizador?
# Actualiza los pesos de la red para minimizar la función de pérdida.
#
# Regla general: peso_nuevo = peso_viejo - learning_rate × gradiente
#
# ¿Por qué Adam?
# Adam es uno de los optimizadores más populares porque:
#
# 1. Adaptive learning rate:
#    Ajusta automáticamente la tasa de aprendizaje para cada parámetro
#    - Parámetros con gradientes grandes → learning rate más pequeño
#    - Parámetros con gradientes pequeños → learning rate más grande
#
# 2. Momentum:
#    "Recuerda" las actualizaciones previas para acelerar el aprendizaje
#    - Ayuda a escapar de mínimos locales
#    - Suaviza el camino hacia la convergencia
#
# 3. Bias correction:
#    Corrige el sesgo de las estimaciones al inicio del entrenamiento
#
# ¿Cómo funciona Adam internamente?
# Mantiene dos "memorias":
# - m (primer momento): promedio móvil de gradientes
# - v (segundo momento): promedio móvil de gradientes al cuadrado
#
# Algoritmo simplificado:
# 1. Calcular gradiente g = ∂Loss/∂peso
# 2. Actualizar m = β₁·m + (1-β₁)·g
# 3. Actualizar v = β₂·v + (1-β₂)·g²
# 4. Corregir bias: m̂ = m/(1-β₁ᵗ), v̂ = v/(1-β₂ᵗ)
# 5. Actualizar peso: w = w - lr·m̂/√(v̂ + ε)
#
# Valores por defecto en PyTorch:
# - β₁ = 0.9 (decaimiento del primer momento)
# - β₂ = 0.999 (decaimiento del segundo momento)
# - ε = 1e-8 (para estabilidad numérica)
#
# modelo.parameters(): ¿Qué es esto?
# Devuelve TODOS los parámetros entrenables del modelo:
# - Pesos de la primera capa lineal (16×8)
# - Bias de la primera capa lineal (8)
# - Pesos de la segunda capa lineal (8×1)
# - Bias de la segunda capa lineal (1)
# Total: 145 parámetros que Adam va a optimizar

# ¿Qué es el learning rate (lr) y por qué 0.01?
# Es el tamaño del paso que damos en cada actualización de pesos.
#
# ¿Por qué 0.01?
# - Es un valor por defecto razonable para Adam
# - Más grande que el usado en el perceptrón (0.1) porque Adam es más estable
# - Valores típicos para Adam: entre 0.001 y 0.01
#
# ¿Qué pasa si cambio el learning rate?
# - lr muy grande (ej: 0.1): Puede diverger, la pérdida sube en lugar de bajar
# - lr muy pequeño (ej: 0.0001): Aprendizaje muy lento, necesita muchas épocas
# - lr óptimo: Depende del problema, se encuentra experimentando
#
# Comparación con perceptrón:
# - Perceptrón: Ajuste manual, tasa fija 0.1
# - MLP con Adam: Ajuste automático, tasa adaptativa basada en 0.01
optimizador = optim.Adam(modelo.parameters(), lr=0.01)

print("Configuración de entrenamiento:")
print(f"  Loss function: Binary Cross Entropy")
print(f"  Optimizador: Adam")
print(f"  Learning rate: 0.01")
print(f"\nAlternativas de optimizadores en PyTorch:")
print(f"  - SGD: Simple pero requiere tuning de lr")
print(f"  - AdaGrad: Bueno para datos sparse")
print(f"  - RMSprop: Precursor de Adam")
print(f"  - Adam: Balance entre velocidad y estabilidad (elegido)")

## 7. Entrenamiento del modelo

Vamos a entrenar el modelo por varias épocas. En cada época:
1. Calculamos las predicciones (forward pass)
2. Calculamos la pérdida
3. Calculamos los gradientes (backpropagation)
4. Actualizamos los pesos (optimizer step)

Este proceso es automático gracias a PyTorch, a diferencia del ajuste manual que hicimos en el perceptrón.

In [ ]:
###############################################################################
# PARÁMETROS DEL ENTRENAMIENTO
###############################################################################

# ¿Por qué 200 épocas?
# Es un número arbitrario pero suficientemente grande para este problema.
#
# ¿Cómo saber cuántas épocas usar?
# - Muy pocas: El modelo no aprende completamente
# - Muchas: Riesgo de overfitting (memoriza los datos de entrenamiento)
# - Ideal: Monitorear la pérdida y parar cuando deje de disminuir
#
# 200 épocas es razonable para:
# - Dataset pequeño (10 ejemplos)
# - Modelo simple (145 parámetros)
# - Optimizador eficiente (Adam)
#
# En la práctica:
# - Usaríamos "early stopping": detener si la pérdida no mejora
# - Dividiríamos los datos en train/validation para monitorear overfitting
epocas = 200

print("="*60)
print("INICIANDO ENTRENAMIENTO")
print("="*60)
print(f"Épocas: {epocas}\n")

###############################################################################
# BUCLE DE ENTRENAMIENTO: GRADIENT DESCENT CON BACKPROPAGATION
###############################################################################
#
# El entrenamiento de redes neuronales sigue estos pasos en cada época:
# 1. Forward pass: calcular predicciones
# 2. Calcular pérdida (loss)
# 3. Backward pass: calcular gradientes (backpropagation)
# 4. Actualizar pesos usando el optimizador
#
# Este proceso es la base del aprendizaje profundo (deep learning)
#
###############################################################################

for epoca in range(epocas):
    ###########################################################################
    # PASO 0: Modo entrenamiento
    ###########################################################################
    # modelo.train() activa el "modo entrenamiento"
    #
    # ¿Por qué es necesario?
    # Algunas capas se comportan diferente en entrenamiento vs evaluación:
    # - Dropout: Se desactiva aleatoriamente neuronas durante entrenamiento,
    #            pero NO durante evaluación
    # - Batch Normalization: Usa estadísticas del batch en train,
    #                        usa estadísticas globales en eval
    #
    # En este modelo simple no afecta (no usamos Dropout ni BatchNorm),
    # pero es buena práctica incluirlo siempre.
    modelo.train()
    
    ###########################################################################
    # PASO 1: Forward pass (Propagación hacia adelante)
    ###########################################################################
    # Calculamos las predicciones del modelo para TODOS los ejemplos a la vez
    #
    # Dimensiones:
    # - X: (10, 16) - 10 frases, 16 features cada una
    # - salida: (10, 1) - 10 predicciones, 1 valor por frase
    #
    # ¿Qué pasa internamente?
    # 1. Primera capa: (10, 16) @ (16, 8) = (10, 8)
    # 2. ReLU: (10, 8) → (10, 8) [elemento a elemento]
    # 3. Segunda capa: (10, 8) @ (8, 1) = (10, 1)
    # 4. Sigmoid: (10, 1) → (10, 1) [elemento a elemento]
    salida = modelo(X)
    
    ###########################################################################
    # PASO 2: Cálculo de la pérdida
    ###########################################################################
    # Comparamos las predicciones con las etiquetas reales
    #
    # criterio(salida, y) calcula:
    # Loss = promedio de BCE para las 10 frases
    # Loss = -1/10 × Σ[yᵢ·log(ŷᵢ) + (1-yᵢ)·log(1-ŷᵢ)]
    #
    # Resultado: Un único número que representa el error del modelo
    # - Loss alta (ej: 2.0): Predicciones malas
    # - Loss baja (ej: 0.1): Predicciones buenas
    # - Loss cercana a 0: Predicciones casi perfectas
    loss = criterio(salida, y)
    
    ###########################################################################
    # PASO 3: Backpropagation (Propagación hacia atrás)
    ###########################################################################
    # Este es el corazón del aprendizaje de redes neuronales
    #
    # ¿Qué hace optimizador.zero_grad()?
    # Limpia los gradientes de la iteración anterior.
    # En PyTorch, los gradientes se ACUMULAN por defecto.
    # Si no los limpiamos, se sumarían a los gradientes nuevos.
    #
    # Analogía: Borrar el pizarrón antes de escribir nuevos cálculos
    optimizador.zero_grad()
    
    # ¿Qué hace loss.backward()?
    # Calcula los gradientes de la pérdida respecto a TODOS los parámetros.
    #
    # Matemáticamente, calcula: ∂Loss/∂w para cada peso w
    #
    # ¿Cómo lo hace?
    # Usa la regla de la cadena del cálculo diferencial:
    # ∂Loss/∂w₁ = ∂Loss/∂salida × ∂salida/∂z₂ × ∂z₂/∂a₁ × ∂a₁/∂z₁ × ∂z₁/∂w₁
    #
    # PyTorch construye un "grafo computacional" durante forward pass
    # y lo recorre en reversa para calcular todos los gradientes automáticamente.
    #
    # Esta es la magia de los frameworks de deep learning:
    # - En el perceptrón calculamos gradientes manualmente
    # - Acá PyTorch lo hace automáticamente con backward()
    #
    # Después de backward(), cada parámetro tiene su atributo .grad actualizado
    # Ej: modelo.net[0].weight.grad contiene ∂Loss/∂W₁
    loss.backward()
    
    ###########################################################################
    # PASO 4: Actualización de pesos
    ###########################################################################
    # optimizador.step() actualiza TODOS los parámetros usando sus gradientes
    #
    # Para cada parámetro w:
    # w_nuevo = w_viejo - learning_rate × gradiente
    #
    # En Adam (nuestro optimizador), la fórmula es más compleja:
    # - Usa momento (promedio de gradientes pasados)
    # - Usa learning rate adaptativo para cada parámetro
    # - Aplica bias correction
    #
    # Resultado: Los 145 parámetros del modelo se ajustan ligeramente
    # para reducir la pérdida en la próxima iteración.
    optimizador.step()
    
    ###########################################################################
    # Monitoreo del progreso
    ###########################################################################
    # Mostramos la pérdida cada 10 épocas para ver cómo aprende el modelo
    if (epoca + 1) % 10 == 0:
        # .item() convierte un tensor de PyTorch a un número Python
        print(f"Época {epoca+1:3d}, Pérdida: {loss.item():.4f}")
        
        # Interpretación de la pérdida:
        # Época 10: ~0.7 → El modelo está aprendiendo
        # Época 100: ~0.3 → Mejorando notablemente
        # Época 200: ~0.05 → Predicciones muy buenas

print("\n" + "="*60)
print("ENTRENAMIENTO FINALIZADO")
print("="*60)
print("\n¿Qué aprendió el modelo?")
print("Los 145 parámetros se ajustaron iterativamente para:")
print("  1. Reconocer patrones de palabras positivas/negativas")
print("  2. Combinar estos patrones de forma no lineal (gracias a ReLU)")
print("  3. Producir probabilidades calibradas (gracias a BCE y Sigmoid)")

## 8. Análisis del entrenamiento

Observá cómo la pérdida disminuye con el tiempo. Esto indica que el modelo está aprendiendo a clasificar mejor las frases.

Una pérdida cercana a 0 significa que el modelo está muy confiado en sus predicciones correctas.

## 9. Evaluación con frases nuevas

Probamos la red con frases que no estaban en el entrenamiento, para ver cómo generaliza.

In [ ]:
frases_prueba = [
    "No me gustó la atención, bastante mala",
    "Muy buena experiencia, todo excelente",
    "Una estafa total, no lo recomiendo",
    "Súper conforme con el servicio",
    "Nada que ver con lo prometido, una decepción"
]

print("="*60)
print("EVALUACIÓN EN FRASES NUEVAS")
print("="*60)

# Vectorizamos las frases de prueba
X_prueba_np = np.array([vectorizar(frase, vocabulario) for frase in frases_prueba], dtype=np.float32)
X_prueba = torch.tensor(X_prueba_np)

# Modo evaluación (desactiva dropout, batch norm, etc.)
modelo.eval()

# Predicción sin calcular gradientes (más eficiente)
with torch.no_grad():
    predicciones = modelo(X_prueba)

# Mostrar resultados
for i, (frase, pred) in enumerate(zip(frases_prueba, predicciones), 1):
    probabilidad = pred.item()
    clase = "Positivo" if probabilidad >= 0.5 else "Negativo"
    print(f"\nFrase {i}: '{frase}'")
    print(f"  Predicción: {clase} (probabilidad: {probabilidad:.2f})")
    
    # Indicador visual de confianza
    if probabilidad >= 0.8 or probabilidad <= 0.2:
        print(f"  Confianza: Alta")
    elif probabilidad >= 0.6 or probabilidad <= 0.4:
        print(f"  Confianza: Media")
    else:
        print(f"  Confianza: Baja (ambiguo)")

print("\n" + "="*60)

## 10. Reflexión final

### ¿Qué aprendimos?

1. **Arquitectura multicapa**: Vimos cómo una red con capas ocultas puede aprender representaciones más complejas que un perceptrón simple.

2. **Activaciones no lineales**: ReLU permite que la red aprenda patrones no lineales, algo imposible con un perceptrón simple.

3. **Entrenamiento automático**: PyTorch maneja automáticamente el cálculo de gradientes (backpropagation) y la actualización de pesos, a diferencia del ajuste manual del perceptrón.

4. **Probabilidades vs decisiones binarias**: La salida Sigmoid nos da una probabilidad (0-1) en lugar de solo 0 o 1, lo que permite medir la confianza del modelo.

### Ventajas sobre el perceptrón simple

- Puede aprender patrones más complejos (no lineales)
- Mejor capacidad de generalización
- Optimización más eficiente con Adam
- Salida probabilística (más informativa)

### Limitaciones que aún persisten

1. **No considera el orden de las palabras**: Bag-of-words sigue sin capturar secuencias
2. **Vocabulario fijo**: Solo conoce palabras predefinidas
3. **Sin contexto global**: Cada palabra se procesa independientemente
4. **Dataset pequeño**: Con solo 10 ejemplos, la generalización es limitada

### ¿Qué sigue?

En la próxima actividad vamos a ver cómo las **redes recurrentes (LSTM)** pueden procesar secuencias de palabras manteniendo memoria del contexto. Esto nos va a permitir:

- Capturar el orden de las palabras
- Entender dependencias temporales
- Procesar frases de longitud variable
- Aprovechar embeddings de palabras

Las LSTM son el paso previo a entender los Transformers, que revolucionaron el NLP.